In [53]:
# Instalar dependencias
!apt-get install -y unixodbc-dev
!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/20.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
!apt-get update
!ACCEPT_EULA=Y apt-get install -y msodbcsql17

# Importación de librerías
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import IntegrityError
from sqlalchemy.orm import sessionmaker
import pymssql

# Autenticación en Google
auth.authenticate_user()

# Autorizar el uso de Google Sheets API
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# ID de la hoja de Google Sheets (extraído de la URL)
sheet_id = '1aY9piTlKM70ikXIYYP0bChmpOsUMgrcyUJ4-ZiO-UKU'

# Abrir el archivo de Google Sheets usando su ID
sheet = gc.open_by_key(sheet_id)

# Seleccionar la hoja que deseas (puedes cambiar el índice 0 por el nombre de la hoja o su índice)
worksheet = sheet.get_worksheet(0)

# Obtener todos los valores como una lista de listas
rows = worksheet.get_all_values()

# Convertir la lista a un DataFrame de pandas (usamos la primera fila como nombres de columnas)
df = pd.DataFrame(rows[1:], columns=rows[0])

# Mostrar los primeros registros del DataFrame para verificar
print(df.head())

# Conexión a la base de datos SQL Server
user = 'sqlserver'
password = 'Contraseña1'
server = '34.135.222.123'
database = 'nbaplayers'

# Crear la cadena de conexión
connection_string = f"mssql+pymssql://{user}:{password}@{server}/{database}"
engine = create_engine(connection_string)

# Crear la sesión
Session = sessionmaker(bind=engine)

# Obtener las fechas existentes en la base de datos
with Session() as session:
    fechas_existentes_query = "SELECT DISTINCT ingestion_date FROM MH2024season"
    fechas_existentes = pd.read_sql(fechas_existentes_query, session.bind)

# Filtrar los nuevos registros basados en las fechas existentes
nuevos_registros = df[~df['ingestion_date'].isin(fechas_existentes['ingestion_date'])]

def insertar_nuevos_registros(df, table_name, engine):
    # Convertir las columnas relevantes a tipo float, manejando errores
    columnas_float = ['PTS_Regular', 'PTS_Playoffs', 'REB_Regular', 'REB_Playoffs',
                      'AST_Regular', 'AST_Playoffs', 'TO_Regular', 'TO_Playoffs',
                      '_3PM_Regular', '_3PM_Playoffs', 'FT_Regular', 'FT_Playoffs']

    for col in columnas_float:
        df[col] = pd.to_numeric(df[col], errors='coerce')  # Convertir a float, poner NaN en caso de error

    # Mostrar datos para depuración
    print("Datos después de la conversión:")
    print(df[columnas_float].head())

    try:
        with engine.begin() as conn:
            df.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"Se insertaron {len(df)} nuevos registros.")
    except IntegrityError as e:
        print(f"Error de integridad: {e}")
    except Exception as e:
        print(f"Error al insertar registros: {e}")

# Verificar si hay nuevos registros para insertar
if not nuevos_registros.empty:
    insertar_nuevos_registros(nuevos_registros, 'MH2024season', engine)
else:
    print("No hay nuevos registros para insertar.")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unixodbc-dev is already the newest version (2.3.11-1).
You might want to run 'apt --fix-broken install' to correct these.
The following packages have unmet dependencies:
 msodbcsql17 : Depends: odbcinst but it is not going to be installed
 unixodbc : Depends: odbcinst1debian2 (>= 2.3.11-1) but it is not going to be installed
            Depends: libodbc1 (>= 2.3.11-1) but it is not going to be installed
 unixodbc-dev : Depends: odbcinst1debian2 (= 2.3.11-1) but it is not going to be installed
E: Unmet dependencies. Try 'apt --fix-broken install' with no packages (or specify a solution).
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d ins